In [1]:
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
torch.manual_seed(1)

In [2]:
math = torch.Tensor([2.3, 9.4, -5.5])
phys = torch.Tensor([2.5, 9.1, 6.4])
me = torch.Tensor([-1., -1., 100])

In [3]:
torch.matmul(math, phys) / (9)

6.232222239176433

In [4]:
torch.matmul(phys, me) / 9

69.82222493489583

In [5]:
word_to_idx = {"hello": 0, "world": 1}

In [6]:
embeds = nn.Embedding(2, 5)

In [7]:
lookup_tensor = torch.LongTensor([word_to_idx["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))

In [8]:
lookup_tensor


 0
[torch.LongTensor of size 1]

In [9]:
hello_embed

Variable containing:
 0.6614  0.2669  0.0617  0.6213 -0.4519
[torch.FloatTensor of size 1x5]

In [10]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 5

In [11]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [12]:
test_sentence[:5]

['When', 'forty', 'winters', 'shall', 'besiege']

In [13]:
trigrams = [ ([test_sentence[i], test_sentence[i+1]], test_sentence[i+2])  
            for i in range(len(test_sentence) - CONTEXT_SIZE)]

In [14]:
trigrams[:3]

[(['When', 'forty'], 'winters'),
 (['forty', 'winters'], 'shall'),
 (['winters', 'shall'], 'besiege')]

In [15]:
vocab = set(test_sentence)

In [16]:
word_to_idx = {word: i for i, word in enumerate(vocab)}

In [17]:
class NGramLanguageModeler(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, context_size):    
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view(1, -1)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

In [18]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [19]:
for epoch in range(10):
    total_loss = torch.Tensor([0])
    
    for context, target in trigrams:
        context_idx = [word_to_idx[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idx))
        model.zero_grad()
        log_prob = model(context_var)
        target_tensor = torch.LongTensor([word_to_idx[target]])
        loss = loss_function(log_prob, autograd.Variable(target_tensor))
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
print(losses)

/home/as/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


[
 521.2977
[torch.FloatTensor of size 1]
, 
 518.9423
[torch.FloatTensor of size 1]
, 
 516.6084
[torch.FloatTensor of size 1]
, 
 514.2942
[torch.FloatTensor of size 1]
, 
 511.9995
[torch.FloatTensor of size 1]
, 
 509.7234
[torch.FloatTensor of size 1]
, 
 507.4640
[torch.FloatTensor of size 1]
, 
 505.2219
[torch.FloatTensor of size 1]
, 
 502.9963
[torch.FloatTensor of size 1]
, 
 500.7864
[torch.FloatTensor of size 1]
]


In [24]:
model.eval()

NGramLanguageModeler(
  (embeddings): Embedding(97, 5)
  (linear1): Linear(in_features=10, out_features=128)
  (linear2): Linear(in_features=128, out_features=97)
)

In [33]:
a = trigrams[0]

In [35]:
context, _ = a

In [36]:
context_idx = [word_to_idx[w] for w in context]

In [38]:
context_idx

[33, 69]

In [42]:
model.eval()

NGramLanguageModeler(
  (embeddings): Embedding(97, 5)
  (linear1): Linear(in_features=10, out_features=128)
  (linear2): Linear(in_features=128, out_features=97)
)

In [43]:
model.embeddings(torch.LongTensor(context_idx))

RuntimeError: save_for_backward can only save input or output tensors, but argument 0 doesn't satisfy this condition